In [1]:
import os
import wandb
import torch
import pandas as pd
import numpy as np
import time
from glob import glob
from torch.utils.data import DataLoader

from delphi.networks.ConvNets import BrainStateClassifier3d
from delphi.utils.datasets import NiftiDataset
from delphi.utils.tools import ToTensor, compute_accuracy, convert_wandb_config, read_config
from sklearn.model_selection import StratifiedShuffleSplit

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
def set_random_seed(seed):
    import random
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    g = torch.Generator()  # can be used in pytorch dataloaders for reproducible sample selection when shuffle=True
    g.manual_seed(seed)

    return g

g = set_random_seed(2020)

In [3]:
def wandb_plots(y_true, y_pred, y_prob, class_labels, dataset):
    wandb.log({
        f"{dataset}-ROC": wandb.plot.roc_curve(y_true=y_true, y_probas=y_prob, labels=class_labels),
        f"{dataset}-PR": wandb.plot.pr_curve(y_true=y_true, y_probas=y_prob, labels=class_labels, ),
        f"{dataset}-ConfMat": wandb.plot.confusion_matrix(y_true=y_true, preds=y_pred, class_names=class_labels)
    })

# Define the classes and data to use

In [4]:
class_labels = sorted(["handleft", "handright", "footleft", "footright", "tongue"])

In [5]:
data_test = NiftiDataset("../t-maps/test", class_labels, 0, device=DEVICE, transform=ToTensor())

# we will split the train dataset into a train (80%) and validation (20%) set.
data_train_full = NiftiDataset("../t-maps/train", class_labels, 0, device=DEVICE, transform=ToTensor())

# we want one stratified shuffled split
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=2020)
idx_train, idx_valid = next(sss.split(data_train_full.data, data_train_full.labels))

data_train = torch.utils.data.Subset(data_train_full, idx_train)
data_valid = torch.utils.data.Subset(data_train_full, idx_valid)

# Set up the sweep

# Define the training function

In [6]:
def train_net(model, config, save_name, logwandb=True):
    
    dl_test = DataLoader(data_test, batch_size=config.batch_size, shuffle=True, generator=g)
    dl_train = DataLoader(data_train, batch_size=config.batch_size, shuffle=True, generator=g)
    dl_valid = DataLoader(data_valid, batch_size=config.batch_size, shuffle=True, generator=g)
    
    best_loss, best_acc = 100, 0
    loss_acc = []
    train_stats, valid_stats = [], []
    patience = 9
    patience_ctr = 0
    
    # loop for the above set number of epochs
    for epoch in range(0, config.epochs):
        _, _ = model.fit(dl_train, lr=config.learning_rate, device=DEVICE)

        # for validating or testing set the network into evaluation mode such that layers like dropout are not active
        with torch.no_grad():
            tloss, tstats = model.fit(dl_train, device=DEVICE, train=False)
            vloss, vstats = model.fit(dl_valid, device=DEVICE, train=False)
                    
        tacc = compute_accuracy(tstats[:, -2], tstats[:, -1])
        vacc = compute_accuracy(vstats[:, -2], vstats[:, -1])

        loss_acc.append(pd.DataFrame([[tloss, vloss, tacc, vacc]],
                                     columns=["train_loss", "valid_loss", "train_acc", "valid_acc"]))
        
        train_stats.append(pd.DataFrame(tstats.tolist(), columns=[*class_labels, *["real", "predicted"]]))
        train_stats[epoch]["epoch"] = epoch
        valid_stats.append(pd.DataFrame(vstats.tolist(), columns=[*class_labels, *["real", "predicted"]]))
        valid_stats[epoch]["epoch"] = epoch
        
        wandb.log({
            "train_acc": tacc, "train_loss": tloss,
            "valid_acc": vacc, "valid_loss": vloss
        })
        
        print('Epoch=%03d, train_loss=%2.3f, train_acc=%1.3f, valid_loss=%2.3f, valid_acc=%1.3f' % 
             (epoch, tloss, tacc, vloss, vacc))
        
        if (vacc >= best_acc) and (vloss <= best_loss):
            # assign the new best values
            best_acc, best_loss = vacc, vloss
            wandb.run.summary["best_valid_accuracy"] = best_acc
            wandb.run.summary["best_valid_epoch"] = epoch
            # save the current best model
            model.save(save_name)
            # plot some graphs for the validation data
            wandb_plots(vstats[:, -2], vstats[:, -1], vstats[:, :-2], class_labels, "valid")
            
            # reset the patience counter
            patience_ctr=0
            
        else:
            patience_ctr+=1
        
        if patience_ctr > patience:
            print('Reached patience. Stopping training and continuing with test set.')
            break

    # save the files
    full_df = pd.concat(loss_acc)
    full_df.to_csv(os.path.join(save_name, "loss_acc_curves.csv"), index=False)
    full_df = pd.concat(train_stats)
    full_df.to_csv(os.path.join(save_name, "train_stats.csv"), index=False)
    full_df = pd.concat(valid_stats)
    full_df.to_csv(os.path.join(save_name, "valid_stats.csv"), index=False)
    
    # EVALUATE THE MODEL ON THE TEST DATA
    with torch.no_grad():
        testloss, teststats = model.fit(dl_test, train=False)
    testacc = compute_accuracy(teststats[:, -2], teststats[:, -1])
    wandb.run.summary["test_accuracy"] = testacc

    wandb.log({"test_accuraccy": testacc, "test_loss": testloss})
    wandb_plots(teststats[:, -2], teststats[:, -1], teststats[:, :-2], class_labels, "test")

    wandb.finish()

# Define the run_train function

In [7]:
# define the training function with the wandb init
def run_train():
    
    # here we initialize weights&biases. 
    with wandb.init() as run:
        # here's the promised conversion of the wandb.config
        # this results into a dict that contains key-value pairs that we can use to configure our network:
        # converted_config['lin_neurons'] = [512, 8, 128]
                
        converted_config = convert_wandb_config(wandb.config, BrainStateClassifier3d._REQUIRED_PARAMS)
                
        model = BrainStateClassifier3d((91, 109, 91), len(class_labels), converted_config)
        
        # We do not necessarily need this line but it is nice to update the config.
        #wandb.config.update(model.config, allow_val_change=True)
        
        t_stamp = time.time()
        save_name = os.path.join("models", f"motor-explo_{t_stamp}")
        wandb.run.name = f"motor-explo-{t_stamp}"
        
        # now train the netwok, yay!
        train_net(model, wandb.config, save_name)

# Run the sweep

In [8]:
sweep_config = read_config("exploration_sweep.yaml")
print(sweep_config)

{'name': 'exploration-sweep', 'entity': 'philis893', 'project': 'thesis', 'method': 'grid', 'metric': {'name': 'valid_acc'}, 'parameters': {'channels1': {'value': 1}, 'channels2': {'value': 8}, 'channels3': {'value': 16}, 'channels4': {'value': 32}, 'channels5': {'value': 64}, 'kernel_size': {'values': [3, 5, 7]}, 'lin_neurons1': {'value': 128}, 'lin_neurons2': {'value': 64}, 'batch_size': {'values': [4, 8, 16, 32]}, 'dropout': {'values': [0.3, 0.4, 0.5, 0.6, 0.7]}, 'learning_rate': {'values': [1e-05, 0.0001, 0.001]}, 'epochs': {'value': 60}}}


In [9]:
# set the wandb sweep config
#os.environ['WANDB_MODE'] = 'offline'
os.environ['WANDB_ENTITY'] = "philis893" # this is my wandb account name. This can also be a group name, for example
os.environ['WANDB_PROJECT'] = "thesis" # this is simply the project name where we want to store the sweep logs and plots
#sweep_id = wandb.sweep(sweep_config)

In [10]:
count=179
wandb.agent("7fysapol", function=run_train, count=count)

wandb: Agent Starting Run: 0kljevar with config:
wandb: 	batch_size: 4
wandb: 	channels1: 1
wandb: 	channels2: 8
wandb: 	channels3: 16
wandb: 	channels4: 32
wandb: 	channels5: 64
wandb: 	dropout: 0.3
wandb: 	epochs: 60
wandb: 	kernel_size: 3
wandb: 	learning_rate: 0.0001
wandb: 	lin_neurons1: 128
wandb: 	lin_neurons2: 64
wandb: Currently logged in as: philis893. Use `wandb login --relogin` to force relogin


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Epoch=000, train_loss=1.386, train_acc=0.271, valid_loss=1.415, valid_acc=0.243
Saving models/motor-explo_1665050978.191694/state_dict.pth


...

0it [00:00, ?it/s]

Epoch=009, train_loss=0.224, train_acc=0.938, valid_loss=0.333, valid_acc=0.864


0it [00:00, ?it/s]

0it [00:00, ?it/s]

wandb: ERROR Error while calling W&B API: could not find agent ubklj37x during agentHeartbeat (<Response [404]>)
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/bubbles/lib/python3.8/site-packages/wandb/sdk/lib/retry.py", line 108, in __call__
    result = self._call_fn(*args, **kwargs)
  File "/opt/anaconda3/envs/bubbles/lib/python3.8/site-packages/wandb/sdk/internal/internal_api.py", line 158, in execute
    return self.client.execute(*args, **kwargs)
  File "/opt/anaconda3/envs/bubbles/lib/python3.8/site-packages/wandb/vendor/gql-0.2.0/wandb_gql/client.py", line 52, in execute
    result = self._get_result(document, *args, **kwargs)
  File "/opt/anaconda3/envs/bubbles/lib/python3.8/site-packages/wandb/vendor/gql-0.2.0/wandb_gql/client.py", line 60, in _get_result
    return self.transport.execute(document, *args, **kwargs)
  File "/opt/anaconda3/envs/bubbles/lib/python3.8/site-packages/wandb/vendor/gql-0.2.0/wandb_gql/transport/requests.p